In [8]:
import nltk
import gensim
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim import corpora, models

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [82]:
from os import listdir
from pprint import pprint
class Corpus(object):
    """
    A collection of documents.
    """
    
    def __init__(self, documents_path):
        """
        Initialize empty document list.
        """
        self.word_tokens = []
        self.documents = []
        self.vocabulary = []
        self.filtered_words = []
        self.documents_path = documents_path
        self.number_of_documents = 0
        self.vocabulary_size = 0
        self.ECallDocuments =[]
    
    def tokenize(self):
        """
        word_tokenize
        """
    
        filenames = listdir(self.documents_path)
        for filename in filenames:
            file_path = f"{self.documents_path}/{filename}"
            str = "";
            with open(file_path, "r") as path:
                lines = path.readlines()
                for line in lines:
                    words = re.split("\t|\n| ", line)
                    for word in words:
                        str += word + " ";
            tokens = word_tokenize(str)
            filtered_tokens = []
            tags = nltk.pos_tag(tokens)
            for word, pos in tags:
                if (pos == 'NN' or pos == 'VERB' and len(word) > 3):
                    self.word_tokens.append(word)
                
            # Append the content to the list
            self.ECallDocuments.append(self.word_tokens) # Build the training corpus 'list of lists'
       
            
       
    
    def preprocess_text(self):
        """
        remove stop words ans stemming
        """
        stop_words = set(stopwords.words("english")) #choose english as the languagse 
        ps = PorterStemmer()
        
        for word in self.word_tokens:
            if word not in stop_words and len(word) > 3:
                self.filtered_words.append(ps.stem(word))
        self.filtered_words_freDis = nltk.FreqDist(self.filtered_words)
        
        print("most common 20", self.filtered_words_freDis.most_common(20))
    
    def lda_bow(self):
        dictionary = corpora.Dictionary(self.ECallDocuments)
        
        #covert tokenized documents into a documents-term matrix
#         bow_corpus = [dictionary.doc2bow(self.filtered_words)] #bag-of-words

        bow_corpus = [dictionary.doc2bow(doc) for doc in self.ECallDocuments] # Apply Bag of Wor
        
        #generate LDA model
        ldamodel = gensim.models.ldamodel.LdaModel(bow_corpus, num_topics = 3, id2word = dictionary, passes = 2)
        
        pprint(ldamodel.print_topics(num_topics=3, num_words=20))
        
        ### GET TOPIC ALLOCATIONS FOR TRAINING CORPUS DOCUMENTS ###
        # Set up Bag of Words and TFIDF
        corpus = [dictionary.doc2bow(doc) for doc in self.ECallDocuments] # Apply Bag of Wor
        doc_no = 0 # Set document counter
        for doc in self.ECallDocuments:
            bof_doc = ldamodel[corpus[doc_no]] # Apply TFIDF model to individual documents
            print(ldamodel.get_document_topics(bof_doc)) # Get and print document topic allocations
            doc_no += 1

        print('-'*50)
        

In [83]:
def main():
    documents_path = 'ami-transcripts'
    corpus = Corpus(documents_path)
    corpus.tokenize()
    corpus.preprocess_text()
    print("filtered_word size: " + str(len(corpus.filtered_words)))
    corpus.lda_bow()

if __name__ == '__main__':
    main()

most common 20 [('yeah', 3446), ('someth', 2807), ('thing', 2164), ('control', 2162), ('kind', 1583), ('remot', 1555), ('time', 1518), ('button', 1427), ('okay', 1384), ('design', 1363), ('sort', 1172), ('meet', 1143), ('idea', 962), ('point', 901), ('market', 790), ('screen', 785), ('colour', 763), ('recognit', 724), ('stuff', 718), ('product', 714)]
filtered_word size: 99366
[(0,
  '0.031*"uh" + 0.026*"something" + 0.021*"control" + 0.020*"thing" + '
  '0.017*"kind" + 0.017*"yeah" + 0.013*"design" + 0.012*"time" + '
  '0.012*"remote" + 0.011*"button" + 0.009*"point" + 0.009*"okay" + '
  '0.008*"way" + 0.008*"bit" + 0.008*"meeting" + 0.008*"sort" + 0.008*"idea" + '
  '0.007*"screen" + 0.007*"stuff" + 0.006*"problem"'),
 (1,
  '0.039*"uh" + 0.029*"yeah" + 0.023*"something" + 0.018*"thing" + '
  '0.015*"control" + 0.014*"remote" + 0.013*"time" + 0.013*"okay" + '
  '0.012*"button" + 0.012*"kind" + 0.011*"design" + 0.010*"meeting" + '
  '0.010*"sort" + 0.010*"bit" + 0.009*"way" + 0.008*"i